# PyTorch

## MeuralNetworks

### Imports

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

ModuleNotFoundError: No module named 'torch'

### Get Device

look if there is a Hardware piece that is faster than cpu, if there is no ``cuda`` device, continue with ``cpu``

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### Define Class

a class that implements ``nn.Module``

in the constructor we define the Layers

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

generating a Neural Network on the ``device``

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Throwing Data into the Model

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


### Layers

#### Visualisation Data

```torch.rand(number of Tensors,height,width)```

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image)

tensor([[[0.8689, 0.2922, 0.5651,  ..., 0.6618, 0.2001, 0.9198],
         [0.1279, 0.7180, 0.9355,  ..., 0.0058, 0.2984, 0.2979],
         [0.6303, 0.4192, 0.8811,  ..., 0.5976, 0.5355, 0.5172],
         ...,
         [0.4356, 0.9552, 0.5440,  ..., 0.6494, 0.7735, 0.2188],
         [0.2217, 0.1371, 0.3976,  ..., 0.2739, 0.9185, 0.7118],
         [0.1773, 0.0068, 0.3929,  ..., 0.3515, 0.2773, 0.8898]],

        [[0.1518, 0.0647, 0.0260,  ..., 0.7522, 0.2692, 0.3349],
         [0.9547, 0.9295, 0.0349,  ..., 0.6013, 0.3013, 0.5257],
         [0.6707, 0.1005, 0.5270,  ..., 0.3189, 0.7705, 0.8066],
         ...,
         [0.4780, 0.3806, 0.1693,  ..., 0.3789, 0.7216, 0.9762],
         [0.5432, 0.6179, 0.3109,  ..., 0.7107, 0.1210, 0.3499],
         [0.4951, 0.3866, 0.5060,  ..., 0.5683, 0.7056, 0.6163]],

        [[0.5409, 0.9445, 0.2620,  ..., 0.2800, 0.1923, 0.4235],
         [0.4468, 0.7430, 0.0717,  ..., 0.7352, 0.3331, 0.9583],
         [0.3564, 0.7579, 0.7932,  ..., 0.0083, 0.5118, 0.

In [ ]:
input_image.size()

torch.Size([3, 28, 28])

#### nn.Flatten

turns 2D data to 1D data

``[1, 1], [2, 2]  -> [1, 1, 2, 2]``

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

linear transformation on the input using its stored weights and biases

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

non-linear activations. These do the complex maping inside the NN. There are other methodes than ReLU to do these actions

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1.size()}")

Before ReLU: tensor([[-0.2590, -0.1492,  0.1040,  0.4091, -0.2248,  0.1214,  0.8750, -0.7570,
          0.2708,  0.2031,  0.5730,  0.5035, -0.0379,  0.5718, -0.0445,  0.0619,
         -0.3146, -0.0050,  0.0068,  0.0600],
        [-0.2092,  0.2261, -0.0668,  0.4373, -0.1037,  0.1276,  0.3900, -0.7503,
          0.4985,  0.0991,  0.6274, -0.0433, -0.0565,  0.1606,  0.1006,  0.0575,
         -0.0439, -0.4375, -0.1561, -0.0273],
        [-0.1348,  0.1268, -0.0680, -0.0829, -0.1884, -0.1336,  0.9184, -0.3688,
          0.0367,  0.2492,  0.4073,  0.1169,  0.0524,  0.5442, -0.1523,  0.1118,
          0.2310,  0.1281,  0.0049,  0.0926]], grad_fn=<AddmmBackward0>)


After ReLU: torch.Size([3, 20])


#### nn.Sequential

a collection of Layertypes

<img title="nn.Sequential" alt="Picture" src="res/seq.PNG" height="300">

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax

Last Layor. Returns an evaluation of Likelyhood of every Output in an Intervall [0,1].

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#### Visualization

<img title="Layers drawn" alt="Picture" src="res/layers.PNG" height="300">

### Model Parameters

every layer has a parameter or a bias

``parameters()`` or ``named_parameters()``

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0252, -0.0097, -0.0082,  ..., -0.0274,  0.0166, -0.0005],
        [-0.0039,  0.0238,  0.0327,  ..., -0.0348,  0.0271, -0.0046]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0044,  0.0103], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0136, -0.0330, -0.0324,  ..., -0.0272, -0.0408, -0.0296],
        [ 0.0357, -0.0342, -0.0177,  ..., -0.0165, -0.0377,  0.0380]],
       device='cuda:0', grad_fn=<Sl